# Import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import f1_score
from sklearn import metrics

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('C:\Classification_Data\climate-change-edsa2020-21/train.csv')
test = pd.read_csv('C:\Classification_Data\climate-change-edsa2020-21/test.csv') 

In [3]:
train

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954
...,...,...,...
15814,1,RT @ezlusztig: They took down the material on ...,22001
15815,2,RT @washingtonpost: How climate change could b...,17856
15816,0,notiven: RT: nytimesworld :What does Trump act...,384248
15817,-1,RT @sara8smiles: Hey liberals the climate chan...,819732


In [4]:
train.isnull().sum()

sentiment    0
message      0
tweetid      0
dtype: int64

In [5]:
test.isnull().sum()

message    0
tweetid    0
dtype: int64

In [6]:
train['sentiment'].value_counts() 

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

# Splitting out the X variable from the target

In [7]:
y = train['sentiment']
X = train['message'] 

# Turning text into something your model can read

In [8]:
vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X)  

# Splitting the training data into a training and validation set

In [9]:
X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=.2,shuffle=True, stratify=y, random_state=15)

# Training the model and evaluating using the validation set

In [10]:
clf = LinearSVC()
clf.fit(X_train, y_train)
clf_pred = clf.predict(X_val) 

# Checking the performance of our model on the validation set

In [11]:
f1_score(y_val, clf_pred, average="macro") 

0.6846529263160679

In [12]:
print(metrics.classification_report(y_val, clf_pred))

              precision    recall  f1-score   support

          -1       0.79      0.45      0.57       259
           0       0.63      0.50      0.56       471
           1       0.79      0.88      0.83      1706
           2       0.78      0.78      0.78       728

    accuracy                           0.77      3164
   macro avg       0.75      0.65      0.68      3164
weighted avg       0.76      0.77      0.76      3164



# Getting our test set ready

In [13]:
testx = test['message']
test_vect = vectorizer.transform(testx)

# Making predictions on the test set and adding a sentiment column to our original test df

In [14]:
y_pred = clf.predict(test_vect) 

In [15]:
test['sentiment'] = y_pred

In [16]:
test.head()

,message,tweetid,sentiment
0,Europe will now be looking to China to make su...,169760,1
1,Combine this with the polling of staffers re c...,35326,1
2,"The scary, unimpeachable evidence that climate...",224985,1
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263,1
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928,0


# Creating an output csv for submission

In [17]:
test[['tweetid','sentiment']].to_csv('testsubmission.csv', index=False) 